# **Real-Time Emotion Detection using EfficientNet B0**
- 48x48 Grayscale Input
-  5 Classes: angry, happy, sad, stressed, neutral

In [1]:
!pip install opencv-python

In [2]:
!pip install tensorflow


In [3]:
import cv2
import numpy as np
import tensorflow as tf
from collections import deque

### CONFIGURATION

In [4]:
MODEL_PATH = r"C:\Users\ASUS\Documents\Docker\Music_Recommendation_System\models\FC211002_Nethmi\final_emotion_model_tuned.h5" # Your trained model
CLASSES = ['angry', 'happy', 'sad', 'stressed', 'neutral']
IMG_SIZE = (48, 48)
SMOOTHING = 5  # Average last 5 predictions

COLORS = {
    'angry': (0, 0, 255),      # Red
    'happy': (0, 255, 0),      # Green
    'sad': (255, 0, 0),        # Blue
    'stressed': (0, 165, 255), # Orange
    'neutral': (128, 128, 128) # Gray
}

###  LOAD MODEL & FACE DETECTOR

In [5]:
print("Loading model...")
model = tf.keras.models.load_model(MODEL_PATH)
print(f"Model loaded! Parameters: {model.count_params():,}")

# FACE DETECTOR
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    )

Loading model...


Model loaded! Parameters: 4,698,452


### PREPROCESSING 

In [6]:
def preprocess_face(face_img):
    """Convert BGR face to 48x48 grayscale normalized for model"""
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY) if len(face_img.shape) == 3 else face_img
    resized = cv2.resize(gray, IMG_SIZE)
    normalized = resized.astype('float32') / 255.0
    return np.expand_dims(np.expand_dims(normalized, axis=-1), axis=0)

### WEBCAM DETECTION

In [7]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam!")

print("\n Real-Time Detection Started!")
print("   Press 'q' to quit\n")

prediction_buffer = deque(maxlen=SMOOTHING)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        # Extract and predict
        face_img = frame[y:y+h, x:x+w]
        preprocessed = preprocess_face(face_img)
        preds = model.predict(preprocessed, verbose=0)[0]
        
        # Smooth predictions
        prediction_buffer.append(preds)
        smoothed = np.mean(prediction_buffer, axis=0) if len(prediction_buffer) > 0 else preds
        
        label_idx = np.argmax(smoothed)
        label = CLASSES[label_idx]
        confidence = smoothed[label_idx] * 100
        color = COLORS[label]
        
        # Draw box
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
        
        # Draw label background
        text = f"{label.upper()} ({confidence:.1f}%)"
        (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        cv2.rectangle(frame, (x, y-th-10), (x+tw, y), color, -1)
        cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    
    # Display
    cv2.imshow("Emotion Detection - Press 'q' to quit", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Detection ended!")


 Real-Time Detection Started!
   Press 'q' to quit

Detection ended!
